In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as pyplot

import sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import KFold

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Lasso and Ridge, Random Forest, Linear

In [25]:
games = pd.read_csv('games.csv.zip')


games.drop(columns = 'GAME_STATUS_TEXT', inplace = True)
games.head()




,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,0.382,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,0.457,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,0.313,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,0.297,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,0.378,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [26]:
#Dataframe containing team info
teams = pd.read_csv('teams.csv')
display(teams.head(5))

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


In [27]:
#Data Preprocessing

team_id = list(teams['TEAM_ID'])
team_name = list(teams['ABBREVIATION'])

hometeamsdf = pd.DataFrame({'HOME_TEAM_ID':team_id, 'HOME_TEAM': team_name})
awayteamsdf = pd.DataFrame({'VISITOR_TEAM_ID':team_id, 'VISITOR_TEAM': team_name})
#Creating DataFrames for Team Name matched up with Team ID

games_merge = pd.merge(pd.merge(games, hometeamsdf, on = 'HOME_TEAM_ID'), awayteamsdf, on = 'VISITOR_TEAM_ID')
#Joining DataFrames to add Team Name Columns using team ID

games_merge = games_merge[games_merge['PTS_home'] > 0]
#Removing rows with 0 stat values

games_merge['home'] = 1
#Assigning home binary variable

games_merge.drop(columns = ['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'TEAM_ID_home', 'TEAM_ID_away'], inplace = True)
games_merge.sort_values(by = 'GAME_DATE_EST', inplace = True)
games_merge.reset_index(inplace = True)
games_merge.drop(columns = 'index', inplace = True)
display(games_merge.head(5))



,GAME_DATE_EST,GAME_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,HOME_TEAM,VISITOR_TEAM,home
0,2003-10-05,10300001,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1,UTA,DAL,1
1,2003-10-06,10300002,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1,MEM,MIL,1
2,2003-10-07,10300010,2003,104.0,0.506,0.677,0.455,26.0,45.0,86.0,0.380,0.852,0.188,19.0,37.0,1,WAS,NYK,1
3,2003-10-07,10300009,2003,101.0,0.467,0.871,0.444,19.0,39.0,82.0,0.368,0.609,0.364,13.0,50.0,1,SAC,LAC,1
4,2003-10-07,10300005,2003,104.0,0.527,0.657,0.429,22.0,33.0,80.0,0.470,0.667,0.333,10.0,37.0,1,POR,HOU,1


In [28]:
#Duplicating each row but switching stats to create Team vs opp instead of home vs away
#Ex: Row 1 = UTA vs DAL
#Now: Row 2 = DAL vs UTA
# With same stats to capture stats for both teams for rolling averages

games = games_merge

games_append_away = games.copy()

games_append_away['home'] = 0
#Assigning value of 0 for 'home' column for away team dataframe

games_append_away.columns = ['GAME_DATE_EST', 'GAME_ID', 'SEASON', 'PTS_away', 'FG_PCT_away',
                              'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 'PTS_home', 'FG_PCT_home', 'FT_PCT_home',
                              'FG3_PCT_home', 'AST_home', 'REB_home' , 'HOME_TEAM_WINS','VISITOR_TEAM', 'HOME_TEAM', 'home']
#Assigning columns to switch home stats with away stats

games_both = pd.concat([games, games_append_away])
#Concatenating away team data rows as main team

games_both.sort_values(by = ['GAME_DATE_EST', 'GAME_ID'], inplace = True)
games_both.reset_index(inplace = True, drop = True)

games_both.rename(columns = {'PTS_home': 'PTS', 'FG_PCT_home': 'FG_PCT', 'FT_PCT_home':'FT_PCT', 'FG3_PCT_home':'FG3_PCT', 'AST_home': 'AST', 'REB_home': 'REB', 'PTS_away': 'PTS_opp',
       'FG_PCT_away': 'FG_PCT_opp', 'FT_PCT_away': 'FT_PCT_opp', 'FG3_PCT_away': 'FG3_PCT_opp', 'AST_away': 'AST_opp', 'REB_away': 'REB_opp',
       'HOME_TEAM_WINS': 'WIN', 'HOME_TEAM': 'TEAM', 'VISITOR_TEAM': 'TEAM_opp'}, inplace = True)
#Renaming columns from Home vs Away to Team vs Opponent

games_both['WIN'] = (games_both['PTS'] > games_both['PTS_opp']).astype(int)
#Recreating logic for WIN column (Duplicated rows for away team still had same WIN value as home team df)

games_both['diff'] = (games_both['PTS'] - games_both['PTS_opp']).astype(int)
#Assigning column for point differential of each game (+ = win, - = loss)

display(games_both.head(5))

,GAME_DATE_EST,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,TEAM,TEAM_opp,home,diff
0,2003-10-05,10300001,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1,UTA,DAL,1,5
1,2003-10-05,10300001,2003,85.0,0.447,0.500,0.250,20.0,38.0,90.0,0.457,0.735,0.143,23.0,41.0,0,DAL,UTA,0,-5
2,2003-10-06,10300002,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1,MEM,MIL,1,11
3,2003-10-06,10300002,2003,94.0,0.427,0.700,0.154,20.0,43.0,105.0,0.494,0.618,0.267,25.0,48.0,0,MIL,MEM,0,-11
4,2003-10-07,10300003,2003,96.0,0.391,0.600,0.444,19.0,37.0,100.0,0.494,0.690,0.667,25.0,52.0,0,DET,CLE,1,-4


In [29]:

def add_target_win(team):
  team['TARGET_WIN'] = team['WIN'].shift(-1)
  return team
#This function displays Result of Next Game in current game row (Win = 1, Loss = 0)

def add_target_diff(team):
  team['TARGET_diff'] = team['diff'].shift(-1)
  return team
#This function displays Point Differential (diff) of Next Game in current game row (+ = Win, - = Loss)

games_both = games_both.groupby('TEAM', group_keys = False).apply(add_target_diff)

games_both = games_both.groupby('TEAM', group_keys = False).apply(add_target_win)
games_both['TARGET_WIN'][pd.isnull(games_both['TARGET_WIN'])] = 2

games_both

Hawks = games_both[(games_both['TEAM'] == 'ATL')]
display(Hawks.head(11))

<positron-console-cell-29>:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


<positron-console-cell-29>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<positron-console-cell-29>:14: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a

,GAME_DATE_EST,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,TEAM,TEAM_opp,home,diff,TARGET_diff,TARGET_WIN
18,2003-10-08,10300011,2003,80.0,0.350,0.750,0.429,18.0,45.0,83.0,0.438,0.571,0.250,21.0,45.0,0,ATL,CLE,1,-3,-5.0,0.0
41,2003-10-29,20300006,2003,83.0,0.398,0.737,0.214,18.0,58.0,88.0,0.324,0.700,0.160,24.0,55.0,0,ATL,NOP,0,-5,-6.0,0.0
77,2003-10-31,20300024,2003,94.0,0.400,0.714,0.583,22.0,48.0,100.0,0.400,0.759,0.500,27.0,53.0,0,ATL,CHI,0,-6,-4.0,0.0
86,2003-11-01,20300029,2003,99.0,0.468,0.677,0.375,23.0,36.0,103.0,0.448,0.568,0.286,19.0,52.0,0,ATL,IND,1,-4,10.0,1.0
112,2003-11-03,20300042,2003,90.0,0.427,0.652,0.333,20.0,50.0,80.0,0.407,0.588,0.222,21.0,42.0,1,ATL,NOP,1,10,-27.0,0.0
149,2003-11-05,20300060,2003,72.0,0.367,0.500,0.333,19.0,43.0,99.0,0.446,0.645,0.278,23.0,52.0,0,ATL,GSW,0,-27,-7.0,0.0
197,2003-11-08,20300084,2003,83.0,0.438,0.786,0.100,21.0,45.0,90.0,0.425,0.900,0.500,28.0,41.0,0,ATL,POR,0,-7,10.0,1.0
207,2003-11-09,20300089,2003,91.0,0.479,0.789,0.533,16.0,41.0,81.0,0.379,0.737,0.056,12.0,46.0,1,ATL,OKC,0,10,-12.0,0.0
233,2003-11-11,20300102,2003,103.0,0.543,0.952,0.389,25.0,30.0,115.0,0.464,0.864,0.462,21.0,45.0,0,ATL,LAC,0,-12,5.0,1.0
247,2003-11-12,20300109,2003,99.0,0.457,0.833,0.476,19.0,44.0,94.0,0.429,0.720,0.222,14.0,41.0,1,ATL,PHX,0,5,-15.0,0.0


In [30]:
games_both.head()

,GAME_DATE_EST,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,TEAM,TEAM_opp,home,diff,TARGET_diff,TARGET_WIN
0,2003-10-05,10300001,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1,UTA,DAL,1,5,7.0,1.0
1,2003-10-05,10300001,2003,85.0,0.447,0.500,0.250,20.0,38.0,90.0,0.457,0.735,0.143,23.0,41.0,0,DAL,UTA,0,-5,10.0,1.0
2,2003-10-06,10300002,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1,MEM,MIL,1,11,-2.0,0.0
3,2003-10-06,10300002,2003,94.0,0.427,0.700,0.154,20.0,43.0,105.0,0.494,0.618,0.267,25.0,48.0,0,MIL,MEM,0,-11,-6.0,0.0
4,2003-10-07,10300003,2003,96.0,0.391,0.600,0.444,19.0,37.0,100.0,0.494,0.690,0.667,25.0,52.0,0,DET,CLE,1,-4,15.0,1.0


In [40]:
#Creating rolling averages for last 10 games stats

def find_team_averages(team):
    numeric_cols = team.select_dtypes(include=['number'])  # Select only numeric columns
    rolling = numeric_cols.rolling(10).mean()
    return rolling

# Calculate rolling averages
df_rolling_avg = games_both.groupby(['TEAM', 'SEASON'], group_keys=False).apply(find_team_averages)

# Drop the overlapping columns before joining
non_numeric_cols = games_both[['GAME_DATE_EST', 'TEAM', 'TEAM_opp']]
df_combined = non_numeric_cols.join(df_rolling_avg)

display(df_combined)


<positron-console-cell-40>:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,GAME_DATE_EST,TEAM,TEAM_opp,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,home,diff,TARGET_diff,TARGET_WIN
0,2003-10-05,UTA,DAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-05,DAL,UTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-06,MEM,MIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-06,MIL,MEM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-07,DET,CLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53099,2022-12-21,CHA,LAC,22200405.9,2022.0,112.9,0.4409,0.7595,0.3187,23.0,46.2,122.8,0.5019,0.7568,0.4175,27.8,46.0,0.1,0.5,-9.9,NaN,0.3
53100,2022-12-22,NOP,SAS,22200404.3,2022.0,118.7,0.4857,0.7964,0.3418,26.4,44.4,116.0,0.4687,0.7500,0.3453,25.8,43.9,0.6,0.6,2.7,NaN,0.7
53101,2022-12-22,SAS,NOP,22200395.6,2022.0,110.4,0.4762,0.7502,0.3343,25.1,43.1,117.0,0.4913,0.7790,0.3484,24.9,43.5,0.4,0.6,-6.6,NaN,0.6
53102,2022-12-22,UTA,WAS,22200416.4,2022.0,114.8,0.4645,0.7415,0.3679,22.9,41.6,116.9,0.5107,0.7736,0.3733,24.8,44.5,0.5,0.6,-2.1,NaN,0.7


In [41]:
rolling_columns = [f"{column}_10" for column in df_rolling_avg.columns]
#Creates list of column names with "_10" added to end (Signifying last 10 game avg)

df_rolling_avg.columns = rolling_columns


df = pd.concat([games_both, df_rolling_avg], axis = 1)
# df = df.drop(columns = ['home_10', 'TARGET_10'], axis = 1)

df.head()

,GAME_DATE_EST,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,TEAM,TEAM_opp,home,diff,TARGET_diff,TARGET_WIN,GAME_ID_10,SEASON_10,PTS_10,FG_PCT_10,FT_PCT_10,FG3_PCT_10,AST_10,REB_10,PTS_opp_10,FG_PCT_opp_10,FT_PCT_opp_10,FG3_PCT_opp_10,AST_opp_10,REB_opp_10,WIN_10,home_10,diff_10,TARGET_diff_10,TARGET_WIN_10
0,2003-10-05,10300001,2003,90.0,0.457,0.735,0.143,23.0,41.0,85.0,0.447,0.500,0.250,20.0,38.0,1,UTA,DAL,1,5,7.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-05,10300001,2003,85.0,0.447,0.500,0.250,20.0,38.0,90.0,0.457,0.735,0.143,23.0,41.0,0,DAL,UTA,0,-5,10.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-06,10300002,2003,105.0,0.494,0.618,0.267,25.0,48.0,94.0,0.427,0.700,0.154,20.0,43.0,1,MEM,MIL,1,11,-2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-06,10300002,2003,94.0,0.427,0.700,0.154,20.0,43.0,105.0,0.494,0.618,0.267,25.0,48.0,0,MIL,MEM,0,-11,-6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-07,10300003,2003,96.0,0.391,0.600,0.444,19.0,37.0,100.0,0.494,0.690,0.667,25.0,52.0,0,DET,CLE,1,-4,15.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df = df.dropna()
df.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'SEASON', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'PTS_opp', 'FG_PCT_opp', 'FT_PCT_opp', 'FG3_PCT_opp', 'AST_opp', 'REB_opp', 'WIN', 'TEAM', 'TEAM_opp', 'home', 'diff',
       'TARGET_diff', 'TARGET_WIN', 'GAME_ID_10', 'SEASON_10', 'PTS_10', 'FG_PCT_10', 'FT_PCT_10', 'FG3_PCT_10', 'AST_10', 'REB_10', 'PTS_opp_10', 'FG_PCT_opp_10', 'FT_PCT_opp_10', 'FG3_PCT_opp_10', 'AST_opp_10',
       'REB_opp_10', 'WIN_10', 'home_10', 'diff_10', 'TARGET_diff_10', 'TARGET_WIN_10'],
      dtype='object')

In [44]:
df[df['TEAM'] == "WAS"].head()

,GAME_DATE_EST,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,TEAM,TEAM_opp,home,diff,TARGET_diff,TARGET_WIN,GAME_ID_10,SEASON_10,PTS_10,FG_PCT_10,FT_PCT_10,FG3_PCT_10,AST_10,REB_10,PTS_opp_10,FG_PCT_opp_10,FT_PCT_opp_10,FG3_PCT_opp_10,AST_opp_10,REB_opp_10,WIN_10,home_10,diff_10,TARGET_diff_10,TARGET_WIN_10
263,2003-11-14,20300117,2003,101.0,0.448,0.679,0.286,16.0,43.0,105.0,0.409,0.700,0.400,19.0,53.0,0,WAS,MIA,0,-4,-24.0,0.0,18300049.2,2003.0,92.8,0.4395,0.7134,0.3308,20.6,44.6,89.4,0.4018,0.7582,0.2838,20.9,43.1,0.4,0.5,3.4,-0.8,0.3
282,2003-11-15,20300127,2003,71.0,0.337,0.556,0.385,13.0,40.0,95.0,0.475,0.789,0.400,28.0,53.0,0,WAS,SAS,1,-24,9.0,1.0,19300060.9,2003.0,89.5,0.4226,0.7013,0.3238,19.3,44.1,90.3,0.4113,0.7519,0.3050,21.8,44.7,0.3,0.5,-0.8,0.6,0.4
311,2003-11-17,20300141,2003,106.0,0.390,0.750,0.375,19.0,44.0,97.0,0.423,0.872,0.231,12.0,39.0,1,WAS,ATL,0,9,11.0,1.0,20300073.2,2003.0,93.0,0.4258,0.7096,0.3228,19.7,44.4,92.4,0.4153,0.7641,0.3081,21.3,44.7,0.4,0.5,0.6,-0.8,0.4
334,2003-11-19,20300153,2003,106.0,0.389,0.774,0.421,18.0,46.0,95.0,0.468,0.778,0.250,22.0,43.0,1,WAS,CLE,1,11,-21.0,0.0,20300087.5,2003.0,93.7,0.4160,0.7203,0.3232,19.0,44.2,94.5,0.4304,0.7806,0.3100,21.9,44.3,0.4,0.6,-0.8,-2.6,0.4
367,2003-11-21,20300169,2003,87.0,0.408,0.658,0.286,17.0,44.0,108.0,0.473,0.824,0.429,38.0,49.0,0,WAS,DEN,0,-21,-3.0,0.0,20300102.5,2003.0,94.5,0.4125,0.7099,0.3375,19.0,44.9,97.1,0.4391,0.7797,0.3196,23.3,44.3,0.4,0.6,-2.6,-1.6,0.4


In [45]:
def next_game_info(team, column_name):
    next_column = team[column_name].shift(-1)
    return next_column


def add_next_game_info(df, column_name):
    return df.groupby('TEAM', group_keys = False).apply(lambda x: next_game_info(x, column_name))

#Functions to display next opponent in current row

df['home_next'] = add_next_game_info(df, 'home')
df['team_opp_next'] = add_next_game_info(df, 'TEAM_opp')
df['game_date_next'] = add_next_game_info(df, 'GAME_DATE_EST')

<positron-console-cell-45>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<positron-console-cell-45>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
<positron-console-cell-45>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False`

In [46]:
df.head(5)

# df[['TEAM', 'TEAM_opp']].head()

,GAME_DATE_EST,GAME_ID,SEASON,PTS,FG_PCT,FT_PCT,FG3_PCT,AST,REB,PTS_opp,FG_PCT_opp,FT_PCT_opp,FG3_PCT_opp,AST_opp,REB_opp,WIN,TEAM,TEAM_opp,home,diff,TARGET_diff,TARGET_WIN,GAME_ID_10,SEASON_10,PTS_10,FG_PCT_10,FT_PCT_10,FG3_PCT_10,AST_10,REB_10,PTS_opp_10,FG_PCT_opp_10,FT_PCT_opp_10,FG3_PCT_opp_10,AST_opp_10,REB_opp_10,WIN_10,home_10,diff_10,TARGET_diff_10,TARGET_WIN_10,home_next,team_opp_next,game_date_next
224,2003-11-11,20300098,2003,125.0,0.581,0.800,0.474,40.0,45.0,97.0,0.481,0.700,0.350,19.0,30.0,1,DAL,NOP,1,28,11.0,1.0,18300039.5,2003.0,96.9,0.4481,0.7493,0.2944,22.9,45.1,92.2,0.4302,0.7292,0.3246,21.8,42.4,0.6,0.4,4.7,6.3,0.7,1.0,HOU,2003-11-13
235,2003-11-12,20300103,2003,89.0,0.431,0.686,0.500,14.0,47.0,82.0,0.329,0.667,0.296,22.0,42.0,1,CHI,BOS,0,7,-3.0,0.0,19300052.2,2003.0,85.2,0.4220,0.7231,0.4243,19.4,41.2,95.0,0.4346,0.6852,0.4416,22.4,42.7,0.4,0.5,-9.8,-9.7,0.4,1.0,MIN,2003-11-13
238,2003-11-12,20300105,2003,97.0,0.452,0.667,0.250,20.0,47.0,107.0,0.446,0.762,0.474,18.0,37.0,0,ORL,MEM,1,-10,-5.0,0.0,18300042.0,2003.0,87.7,0.4031,0.7159,0.2971,17.3,38.3,96.7,0.4578,0.7938,0.3898,20.9,45.2,0.1,0.6,-9.0,-8.5,0.1,0.0,DEN,2003-11-14
241,2003-11-12,20300106,2003,83.0,0.451,0.567,0.250,21.0,42.0,88.0,0.455,0.765,0.385,21.0,40.0,0,CLE,MIA,0,-5,-9.0,0.0,18300048.3,2003.0,91.3,0.4548,0.6842,0.3672,24.2,45.9,93.1,0.4316,0.7285,0.3469,21.1,39.9,0.4,0.3,-1.8,-3.1,0.3,0.0,BOS,2003-11-14
242,2003-11-12,20300107,2003,96.0,0.463,0.786,0.500,24.0,30.0,89.0,0.486,0.783,0.083,26.0,35.0,1,NOP,UTA,1,7,22.0,1.0,19300051.7,2003.0,98.9,0.4616,0.7361,0.4130,23.8,38.9,97.6,0.4671,0.7859,0.2801,23.8,40.9,0.7,0.6,1.3,2.8,0.7,1.0,POR,2003-11-15


In [47]:
#Merging Data of Opponent last 10 games

final = df.merge(df[rolling_columns + ['team_opp_next', 'TEAM', 'game_date_next']], left_on = ['TEAM', 'game_date_next'], right_on = ['team_opp_next', 'game_date_next'])
#Merging team stats with team opponent stats using next game date

final.columns
#Columns with x are "TEAM"
# Columns with y are Opponent
final[['TEAM_x', 'TEAM_y', 'home_next']]

,TEAM_x,TEAM_y,home_next
0,ATL,BKN,1.0
1,BOS,NYK,0.0
2,CLE,PHI,1.0
3,BKN,ATL,0.0
4,WAS,SAS,1.0
...,...,...,...
47371,GSW,NYK,0.0
47372,MIN,DAL,1.0
47373,CHA,SAC,0.0
47374,LAL,PHX,0.0


In [48]:
object_columns = list(final.columns[final.dtypes == "object"])
object_columns


removed_columns = ['TARGET_WIN']
removed_columns = object_columns + removed_columns

final.columns

# selected_columns = final.columns[~final.columns.isin(removed_columns)]

# rfe.fit(final[selected_columns], final['TARGET_WIN'])

# selected_columns = full.columns[~full.columns.isin(removed_columns)]
# sfs.fit(full[selected_columns], full["target"])

Index(['GAME_DATE_EST', 'GAME_ID', 'SEASON', 'PTS', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'AST', 'REB', 'PTS_opp', 'FG_PCT_opp', 'FT_PCT_opp', 'FG3_PCT_opp', 'AST_opp', 'REB_opp', 'WIN', 'TEAM_x', 'TEAM_opp', 'home', 'diff',
       'TARGET_diff', 'TARGET_WIN', 'GAME_ID_10_x', 'SEASON_10_x', 'PTS_10_x', 'FG_PCT_10_x', 'FT_PCT_10_x', 'FG3_PCT_10_x', 'AST_10_x', 'REB_10_x', 'PTS_opp_10_x', 'FG_PCT_opp_10_x', 'FT_PCT_opp_10_x', 'FG3_PCT_opp_10_x',
       'AST_opp_10_x', 'REB_opp_10_x', 'WIN_10_x', 'home_10_x', 'diff_10_x', 'TARGET_diff_10_x', 'TARGET_WIN_10_x', 'home_next', 'team_opp_next_x', 'game_date_next', 'GAME_ID_10_y', 'SEASON_10_y', 'PTS_10_y', 'FG_PCT_10_y',
       'FT_PCT_10_y', 'FG3_PCT_10_y', 'AST_10_y', 'REB_10_y', 'PTS_opp_10_y', 'FG_PCT_opp_10_y', 'FT_PCT_opp_10_y', 'FG3_PCT_opp_10_y', 'AST_opp_10_y', 'REB_opp_10_y', 'WIN_10_y', 'home_10_y', 'diff_10_y', 'TARGET_diff_10_y',
       'TARGET_WIN_10_y', 'team_opp_next_y', 'TEAM_y'],
      dtype='object')

## Writing Data to CSV

In [50]:
final.to_csv("NBA-Game-Data-n=10.csv", index = False)